** Some imports **

In [1]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
%matplotlib inline

** Load Data ** 

In [2]:
names = ['ind', 'ax', 'ay', 'az', 'label']
acc_names = ['ax', 'ay', 'az']
label2str = {1:'Working at Computer', 2:'Standing Up, Walking and Going up-down stairs', 
             3:'Standing', 4:'Walking',5:'Going Up\Down Stairs', 6:'Walking and Talking with Someone', 
             7:'Talking while Standing'}
df = pd.read_csv("Dataset/1.csv", sep=',', names=names)
del df['ind']
df = df[df.label != 0]
df['label_str'] = df.label.apply(lambda x:label2str[x])

In [6]:
df.groupby('label').cov()

ax           ay           az
label                                          
1     ax   766.912701   358.793506   159.954482
      ay   358.793506   471.341038    63.497769
      az   159.954482    63.497769   544.958617
2     ax  1098.241635  -346.474064    34.541232
      ay  -346.474064  2549.600398    23.695825
      az    34.541232    23.695825  2364.185079
3     ax   379.402989   -21.247176   266.515652
      ay   -21.247176   555.341473    73.752778
      az   266.515652    73.752778  1080.162671
4     ax  2102.961896 -1533.919020   442.949673
      ay -1533.919020  7766.570178   513.316861
      az   442.949673   513.316861  2378.773895
5     ax  2477.400794  -769.475040   767.770484
      ay  -769.475040  7875.379841  1188.936279
      az   767.770484  1188.936279  5649.506835
6     ax  2179.244137 -1367.891331   923.617390
      ay -1367.891331  6152.073810   379.497760
      az   923.617390   379.497760  3485.538537
7     ax   416.227573    37.649830   202.468011
      ay    37.649830    83.208652   -54.644529
      az   202.468011   -54.644529  1556.275047

** Add signals ** 

In [ ]:
# Magnitude
df['mag'] = np.sqrt(np.square(df[acc_names]).sum(axis=1)) 
# Median filter - 3rd ordre
def med_fil(df, names):
    df_r = pd.DataFrame()
    df = df[names]
    for column in df.columns:
        name = column+'_mf'
        df_r[name] = signal.medfilt(df[column].values)
    return df_r
df_med = med_fil(df, acc_names)
# Diffrential
def diffrential(df, names):
    df = df[names]
    df_r = df.diff(periods=1, axis=0).fillna(method='backfill')
    df_r.columns = [names[0]+'_d', names[1]+'_d', names[2]+'_d']
    return df_r
df_diff = diffrential(df, acc_names)
# Low pass filter
def lowpass(df, names):
    df = df[names]
    df_r = pd.DataFrame()
    fs = 52 # frequence sampling is 52
    f_cut = 1 # cutoff frequency
    fs_n = f_cut*2.0/fs # normalized frequency
    b,a = signal.butter(N=3, Wn=fs_n, btype='low')
    for column in df.columns : 
        name = column+'_lp'
        df_r[name] = signal.lfilter(b,a,df[column].values)
    return df_r
df_lp = lowpass(df, acc_names)  
# High pass filter 
def highpass(df, names):
    df = df[names]
    df_r = pd.DataFrame()
    fs = 52 # frequence sampling is 52
    f_cut = 1 # cutoff frequency
    fs_n = f_cut*2.0/fs # normalized frequency
    b,a = signal.butter(N=3, Wn=fs_n, btype='high')
    for column in df.columns : 
        name = column+'_hp'
        df_r[name] = signal.lfilter(b,a,df[column].values)
    return df_r
df_hp = highpass(df, acc_names)

# Total
df = pd.concat([df, df_med, df_diff, df_lp, df_hp], axis=1)
df.head()

In [ ]:
names = ['ax','ay','az','mag','ax_mf','ay_mf','az_mf','ax_lp','ay_lp','az_lp', 'ax_hp','ay_hp','az_hp']


def extract_variables(df):
    def rms(df, names):
        # TODO
        pass
    def minmax(df, names=names):
        res = []
        for column in df.columns:   
            if column not in  :
                i_min = df[column].argmin()
                i_max = df[column].argmax()
                i1 = min(i_min, i_max)
                i2 = max(i_min, i_max)
                m = df[column].values[i1:i2].mean()
                res.append(m)
        return res
        
    m = df.mean(axis=0).values
    ma = df.mad(axis=0).values
    std = df.std(axis=0).values
    minimum = df.min(axis=0).values
    maximum = df.max(axis=0).values
    skew = df.skew(axis=0).values
    kurt = df.kurtosis(axis=0).values
#     minmax = minmax(df, names) 
#     rms = rms(df, names) # TODO
    inteQ = (df.quantile(q=0.75) - df.quantile(q=0.25)).values
    r = np.hstack([m, ma, std, minimum, maximum, skew, kurt, inteQ])    
    return r

### Windowing

In [ ]:
def windowing(signal,size,step):
    d = len(signal) #length of the signal
    nk = int(np.floor((d-size+1)/step))+1 #le nombre de fenetres
    wk = np.zeros((nk,size)) #windows
    for j in range(nk):
        wk[j,:] = signal[j*step:j*step+size]
    return wk

In [ ]:
def window_labels(labels,size,step):
    d = len(labels) #length of the signal
    nk = int(np.floor((d-size+1)/step))+1 #le nombre de fenetres
    labelwk = np.zeros((nk)) #window labels
    for j in range(nk):
        labelwk[j] = np.max(np.argmax(np.bincount(labels[j*step:j*step+size])))
    return labelwk

In [ ]:
def extract_windows(df,size,step):
    
    """
    extract windows with the specified size and step from the dataframe df
    
    Returns:
    L : List of dataframes. Each dataframe contains a window extracted from each signal in df.
    labels: labels of windows
    """
    
    L = []
    n = df.shape[0]
    L_windows = dict()
    n_windows = int(np.floor((n-size+1)/step))+1
    for column in df.columns:
        if column not in ['label','label_str']:
            L_windows[column] = windowing(df[column],size,step)
    for i in range(n_windows):
        ddf = pd.DataFrame()
        for column in df.columns:
            if column not in ['label','label_str']:
                ddf[column] = L_windows[column][i,:]
        L.append(ddf)
    labels = window_labels(df['label'],size,step)
    
    return L,labels

In [ ]:
Nbr_samples = 52
percentage = 0.5 # Between 0 & 1
df_X, df_Y = extract_windows(df,Nbr_samples,percentage*Nbr_samples)

** Extract Matrix of Features ** 

In [ ]:
X = extract_variables(df_X[0])
for i in range(1,len(df_X)):
    vec = extract_variables(df_X[i])
    X = np.vstack([X,vec])
    
y = np.array(df_Y) 



** Machine Learning **

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
X = scale(X)
knn.fit(X,y)
print('training score: '+str(knn.score(X,y)))

In [ ]:
confusion_matrix(y, knn.predict(X))

In [ ]:
lrcv = LogisticRegressionCV(Cs=10, cv=None, penalty='l1', solver='liblinear', n_jobs=8, multi_class='ovr')
lrcv.fit(X,y)
print('training score: '+str(lrcv.score(X,y)))

In [ ]:
param_grid = {'n_neighbors':[3,5,10,15,20,25]}
knncv = GridSearchCV(estimator=knn, param_grid=parm_grid)
knncv.fit(X,y)

In [ ]:
print(knncv.cv_results_)
print(knncv.get_params())

In [ ]:
from sklearn.model_selection import cross_val_score
clf = KNeighborsClassifier(n_neighbors=5)
print(cross_val_score(clf, X, y, cv=5, scoring='accuracy'))

# KDA